Setting SmugMug Print Size Keywords with Jupyter and Python
=========================================

![](jupysmug.png)

## Why am I doing this?

Many years ago I wrote a little J verb `smuprintsizes` that computed 
the [largest standard SmugMug print sizes](https://analyzethedatanotthedrivel.org/2010/02/21/assigning-smugmug-print-size-keys/) when given image dimensions 
and the desired [DPI](https://en.wikipedia.org/wiki/Dots_per_inch). I used 
the output of this verb to set aspect ratio keywords for my SmugMug pictures until changes to 
SmugMug, particularly the introduction of [OAuth authentication](https://en.wikipedia.org/wiki/OAuth), broke my little SmugMug API application that called `smugprintsizes`. 

My print size keyword setter broke years ago but many of these keys still show up in my "top ten" keywords. 

    10x15 4x5 4x6 5x5 5x6.7 5x7 ...
    
Print size keywords were very handy. They made it easy to select paper sizes for one or hundreds of pictures. 
This notebook will use the SmugMug API and Python to compute and set print size keywords.

## The Print Sizes Table

`smugprintsizes` made use of the following table.

     ┌─────┬─────────┬──────────────┐
     │0.7  │17.5 70  │3.5x5 7x10    │
     ├─────┼─────────┼──────────────┤
     │0.8  │20 80    │4x5 8x10      │
     ├─────┼─────────┼──────────────┤
     │0.755│21.2 84.8│4x5.3 8x10.6  │
     ├─────┼─────────┼──────────────┤
     │0.665│24 96    │4x6 8x12      │
     ├─────┼─────────┼──────────────┤
     │0.5  │32 50 128│4x8 5x10 8x16 │
     ├─────┼─────────┼──────────────┤
     │1    │25 64 100│5x5 8x8 10x10 │
     ├─────┼─────────┼──────────────┤
     │0.745│33.5     │5x6.7         │
     ├─────┼─────────┼──────────────┤
     │0.715│35       │5x7           │
     ├─────┼─────────┼──────────────┤
     │0.165│150      │5x30          │
     ├─────┼─────────┼──────────────┤
     │0.4  │160      │8x20          │
     ├─────┼─────────┼──────────────┤
     │0.775│93.5     │8.5x11        │
     ├─────┼─────────┼──────────────┤
     │0.75 │108      │9x12          │
     ├─────┼─────────┼──────────────┤
     │0.77 │130      │10x13         │
     └─────┴─────────┴──────────────┘
     
The first column is the Short/Long image aspect ratio rounded to 0.005. The middle column 
lists areas in square inches of the corresponding print sizes in the last column.

Finding the largest DPI dependent print size is simple matter of:

1. Divide the short image dimension by the long image dimension and round to 0.005.
   This is the aspect ratio.

2. Search for an aspect ratio match in the first column. Many images will not match.
   Quit and return `0z0` for no aspect match.

3. If a match is found compute the print area required for a given DPI and round to 0.5.

4. Find the index of the largest area in the second column that is greater than or equal to the required 
   area computed in the previous step. If there are not enough pixels no area will meet this criterion.
   Quit and return `1z1` for not enough pixels.
   
5. If an area is found select and return the corresponding print size in the last column.



## Computing DPI Dependent Print Area

The use of the print size table is clear with the exception of computing the print area required for a given DPI.
`dpi_area` computes DPI dependent print area.

In [14]:
def round_to(n, precision):
    correction = 0.5 if n >= 0 else -0.5
    return int( n/precision+correction ) * precision

def aspect_ratio(height, width, *, precision=0.005):
    return round_to( min(height, width) / max(height, width), precision )

def dpi_area(height, width, *, dpi=360, precision=0.5):
    return round_to( (height * width) / dpi ** 2, precision )

# image pixel dimensions - order is immaterial
height, width = 2389 , 3344

print('aspect ratio %s' % aspect_ratio(height, width))
print('area at 360 dpi %s' % dpi_area(height, width))
print('area at 720 dpi %s' % dpi_area(height, width, dpi=720))

aspect ratio 0.715
area at 360 dpi 61.5
area at 720 dpi 15.5


In [26]:
aspect_ratios = [0.7, 0.8, 0.755, 0.665, 0.5, 1, 0.745, 0.715, 0.165, 0.4, 0.775, 0.75, 0.77]
print(aspect_ratios)
print(len(aspect_ratios))

[0.7, 0.8, 0.755, 0.665, 0.5, 1, 0.745, 0.715, 0.165, 0.4, 0.775, 0.75, 0.77]
13


In [31]:
print_areas = [[17.5,70],[20,80],[21.2,84.8],[24,96],[32,50,128],[25,64,100],[33.5],[35],[150 ],[160],[93.5],[108 ],[130]]
print(print_areas)
print(len(print_areas))

[[17.5, 70], [20, 80], [21.2, 84.8], [24, 96], [32, 50, 128], [25, 64, 100], [33.5], [35], [150], [160], [93.5], [108], [130]]
13


In [40]:
size_keywords = [['3.5x5','7x10'],['4x5','8x10'],['4x5.3','8x10.6'],
                 ['4x6','8x12'],['4x8','5x10', '8x16'],['5x5','8x8','10x10'],['5x6.7'],
                 ['5x7'],['5x30'],['8x20'],['8.5x11'],['9x12'],['10x13']]
print(size_keywords)
print(len(size_keywords))

[['3.5x5', '7x10'], ['4x5', '8x10'], ['4x5.3', '8x10.6'], ['4x6', '8x12'], ['4x8', '5x10', '8x16'], ['5x5', '8x8', '10x10'], ['5x6.7'], ['5x7'], ['5x30'], ['8x20'], ['8.5x11'], ['9x12'], ['10x13']]
13


In [69]:
size_keywords[5][2-2]

'5x5'

In [86]:
def print_size_key(height, width, *, no_ratio='0z0', no_pixels='1z1', min_area= 16):
    print_ratio = aspect_ratio(height, width)
    print_area = dpi_area(height, width)
    
    # we need larger areas to print 
    if print_area <= min_area:
        print('not enough pixels - area is %s' % print_area)
        return no_pixels
    
    print_key = no_ratio
    print('ratio %s area %s key %s' % (print_ratio, print_area, print_key))
    for i, aspect in enumerate(aspect_ratios):
        if print_ratio == aspect:
            print_key = no_pixels
            
            # not enough or more than enough area
            if print_area < print_areas[i][0]:
                break
            elif print_area > print_areas[i][-1]:
                print_key = size_keywords[i][-1]
                break     
            
            for j, area in enumerate(print_areas[i]):
                print('area %s' % area)
                if area >= print_area and 0 < j:
                    print_key = size_keywords[i][j - 1]
                    break
                    
    return print_key
    
# many sizes available for aspect ratio 1.0
print(print_size_key(3800, 3800))
print(print_size_key(3000, 3000))
print(print_size_key(2000, 2000))
print(print_size_key(500,500))
print(print_size_key(10,10))   

ratio 1.0 area 111.5 key 0z0
10x10
ratio 1.0 area 69.5 key 0z0
area 25
area 64
area 100
8x8
ratio 1.0 area 31.0 key 0z0
area 25
area 64
5x5
not enough pixels - area is 2.0
1z1
not enough pixels - area is 0.0
1z1


In [87]:
# no ratio 
print_size_key(3255, 4119)

ratio 0.79 area 103.5 key 0z0


'0z0'

An image with dimensions of 2389 x 3344 has enough pixels to make a standard 5x7 inch 360 DPI print. 
It does not  have enough pixels to make a 5x7 inch 720 DPI print. 

Print resolution is a hot button issue for photographers. How many dots or pixels are 
required depends on many factors, viewing distance, illumination, image colors, paper gloss and so on. 
Human vision tests have demonstrated that young people with excellent eyesight can tell the difference
between 500 DPI and 600 DPI prints. Resolutions beyond 600 DPI are mostly wasted unless you are using loupes or microscopes.
[According to Dr. Optoglass](https://wolfcrow.com/blog/notes-by-dr-optoglass-the-resolution-of-the-human-eye/):

>*If the average reading distance is 1 foot (12 inches = 305 mm), p @0.4 arc minute is 35.5 microns or about 720 ppi/dpi. p @1 arc minute is 89 microns or about 300 dpi/ppi. This is why magazines are printed at 300 dpi – it’s good enough for most people. Fine art printers aim for 720, and that’s the best it need be. Very few people stick their heads closer than 1 foot away from a painting or photograph.*

Digital printers  complicate DPI issues by applying  sophisticated resizing algorithms that can turn low resolution 
originals into plausible higher resolution copies. I've found that 360 DPI is a good starting point for SmugMug prints.
For exceptional images you can simply divide the 360 DPI image dimensions by two for 720 DPI printing. 

## Next on the Agenda!

Remember how "no good dead goes unpunished." Well, running code will be "enhanced" whether it's necessary or not. 
Now that I have a local directories that contain relevant SmugMug metadata in an easily consumed `CSV` form other notebooks will use these directories to generate what I call "long duration documents."  My prefered long duration sources are `Markdown` and `LaTex`. Both of these text formats will be readable for centuries if printed on high quality acid free paper and stored in numerous "secure and undisclosed locations."

Remember, always [Analyze the Data not the Drivel](https://analyzethedatanotthedrivel.org/).

John Baker, Meridian Idaho